## Making datasets and Dataloaders

In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import pickle
import pandas as pd
import numpy as np

In [2]:
TRAIN_DATA = "data/train_encoded.pickle"
VALID_DATA = "data/valid_encoded.pickle"
TEST_DATA = "data/test_encoded.pickle"

In [3]:
with open(TRAIN_DATA, "rb") as file:
    train_df = pickle.load(file)
    train_df.sort_values(by='Date', inplace=True)
    
    
with open(VALID_DATA, "rb") as file:
    valid_df = pickle.load(file)
    valid_df.sort_values(by='Date', inplace=True)

In [4]:
train_df.head()

,User,Card,Amount,Use Chip,MCC,Errors,IsFraud_target,Date,Outcome,Time_diff,...,Month_sin,Month_cos,Day_sin,Day_cos,Dow_sin,Dow_cos,Hour_sin,Hour_cos,Minute_sin,Minute_cos
0,791,0,68.000000,9,12,121,1,1991-01-02 07:10:00,129,0,...,0.5,0.866025,0.394356,0.918958,0.974928,-0.222521,0.965926,-0.258819,8.660254e-01,0.500000
1,791,0,-68.000000,9,12,121,1,1991-01-02 07:17:00,130,420,...,0.5,0.866025,0.394356,0.918958,0.974928,-0.222521,0.965926,-0.258819,9.781476e-01,-0.207912
2,791,0,113.620003,9,12,121,1,1991-01-02 07:21:00,129,240,...,0.5,0.866025,0.394356,0.918958,0.974928,-0.222521,0.965926,-0.258819,8.090170e-01,-0.587785
3,791,0,114.730003,9,13,121,1,1991-01-02 17:30:00,129,36540,...,0.5,0.866025,0.394356,0.918958,0.974928,-0.222521,-0.965926,-0.258819,5.665539e-16,-1.000000
4,791,0,251.710007,9,14,121,1,1991-01-03 09:03:00,129,55980,...,0.5,0.866025,0.571268,0.820763,0.433884,-0.900969,0.707107,-0.707107,3.090170e-01,0.951057


In [6]:
cat_columns = ["Card", "Use Chip", "MCC", "Errors", "Outcome", "is_diff_merchant",
               "is_diff_merchant_city", "is_diff_merchant_state"]
target_columns = ["IsFraud_target"]
drop_columns = ["User", "Date"]
num_columns = np.setdiff1d(train_df.columns.tolist(), cat_columns+target_columns+drop_columns).tolist()

assert len(train_df.columns) == len(cat_columns+target_columns+drop_columns+num_columns)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, df, cat_columns, num_columns, target_columns, drop_columns, max_latest_seq=None):
        self.df = df
        self.cat_columns = cat_columns
        self.num_columns = num_columns
        self.target_columns = target_columns
        self.drop_columns = drop_columns
        self.max_latest_seq = max_latest_seq
        self.indx_to_user = {i: user for i, user in enumerate(self.df.User.unique())}
    
    def __len__(self):
        return len(self.indx_to_user)
    
    def __getitem__(self, indx):
        user_id = self.indx_to_user[indx]
        user_data = (self.df.loc[self.df.loc[:, 'User']==user_id, :]
                     .drop(columns=self.drop_columns)
                     .reset_index(drop=True))
        
        if self.max_latest_seq:
            if len(user_data)>self.max_latest_seq:
                user_data = user_data[-self.max_latest_seq:]
                
        cat_data = user_data[self.cat_columns].to_numpy()
        num_data = user_data[self.num_columns].to_numpy()
        target = np.unique(user_data[self.target_columns].to_numpy())[0]
        return cat_data, num_data, target

In [8]:
def collate_fn(batch):
    category_data = []
    numerical_data = []
    target_data = []
    for category, numerical, target in batch:
        category_data.append(torch.LongTensor(category))
        numerical_data.append(torch.Tensor(numerical))
        target_data.append(target)
        
    category_data = torch.nn.utils.rnn.pad_sequence(category_data, batch_first=True, padding_value=137)
#     numerical_data = torch.nn.utils.rnn.pad_sequence(numerical_data, batch_first=True, padding_value=0)

    target_data = torch.LongTensor(target_data)
    
    return category_data, numerical_data, target_data

## LSTM settings

In [130]:
n_numerical_col = len(num_columns)  # Constant
NUM_UNIQ_EMBEDDINGS = 138  # Constant

MAX_LATEST_SEQ = 32
BATCH_SIZE = 50
EMBEDDING_DIM = 5
feature_dim = n_numerical_col+len(cat_columns)*EMBEDDING_DIM
N_LSTM_LAYER = 2
HIDDEN_DIM = 128
DROPOUT = 0.2
BIDIRECTIONAL = True

## Data preprocessing

In [131]:
train_dataset = CustomDataset(train_df, cat_columns=cat_columns,
                              num_columns=num_columns,
                              target_columns=target_columns,
                              drop_columns=drop_columns,
                              max_latest_seq=MAX_LATEST_SEQ)

valid_dataset = CustomDataset(valid_df, cat_columns=cat_columns,
                              num_columns=num_columns,
                              target_columns=target_columns,
                              drop_columns=drop_columns,
                              max_latest_seq=MAX_LATEST_SEQ)

train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=2,
                          collate_fn=collate_fn)

valid_loader = DataLoader(valid_dataset, 
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=2,
                          collate_fn=collate_fn)

### Default predictions

In [158]:
y_true = torch.cat([y for cat, num, y in train_loader])
y_hat = torch.zeros_like(y_true)
constant_accuracy = (y_true==y_hat).sum()/len(y_true)
print(f"Constant prediction accuracy: {constant_accuracy}")

Constant prediction accuracy: 0.602142870426178


In [159]:
y_hat = torch.randint(0, 1, (1400,))
random_accuracy = (y_true==y_hat).sum()/len(y_true)
print(f"Random prediction accuracy: {random_accuracy}")

Random prediction accuracy: 0.602142870426178


### RNN Model definition

In [138]:
class RNN_network(nn.Module):
    def __init__(self,
                num_uniq_embeddings,
                embedding_dim,
                feature_dim,
                n_lstm_layer,
                hidden_dim,
                dropout,
                bidirectional,
                n_numerical_col):
        super(RNN_network, self).__init__()
        
        self.num_uniq_embeddings=num_uniq_embeddings
        self.embedding_dim=embedding_dim
        self.feature_dim=feature_dim
        self.n_lstm_layer=n_lstm_layer
        self.hidden_dim=hidden_dim
        self.dropout=dropout if n_lstm_layer>1 else 0
        self.bidirectional=bidirectional
        self.n_numerical_col=n_numerical_col
        
        self.embedding = nn.Embedding(self.num_uniq_embeddings, self.embedding_dim)
        self.batch_norm = nn.BatchNorm1d(self.n_numerical_col)
        self.rnn = nn.LSTM(input_size=self.feature_dim,
                          num_layers=self.n_lstm_layer,
                          hidden_size=self.hidden_dim,
                          batch_first=True,
                          dropout=self.dropout,
                          bidirectional=self.bidirectional)
        
        self.linear_one = nn.Linear(self.hidden_dim*(self.bidirectional+1), self.hidden_dim)
        self.batchnorm_linear = nn.BatchNorm1d(self.hidden_dim)
        self.linear_two = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, batch):
        cat, num = batch
        
        # Concat embeddings
        cat = self.embedding(cat)
        all_but_last_two_dims = cat.size()[:-2]
        cat = cat.view(*all_but_last_two_dims, -1)
        
        # Batchnorm across numeric features and pack it to tensor
        num = [self.batch_norm(numeric) for numeric in num]
        num = torch.nn.utils.rnn.pad_sequence(num, batch_first=True, padding_value=0)
        
        # Concat all features
        batch = torch.cat((num, cat), dim=-1)
        
        out, (hidden, cell) = self.rnn(batch)
        
        if self.bidirectional:
            to_classifier = torch.cat((hidden[-2], hidden[-1]), dim=-1)
        else:
            to_classifier = hidden[-1]
            
        x = self.linear_one(to_classifier)
        x = self.batchnorm_linear(x)
        output = self.linear_two(x)
        return output

In [139]:
model = RNN_network(num_uniq_embeddings=NUM_UNIQ_EMBEDDINGS,
               embedding_dim=EMBEDDING_DIM,
               feature_dim=feature_dim,
               n_lstm_layer=N_LSTM_LAYER,
               hidden_dim=HIDDEN_DIM,
               dropout=DROPOUT,
               bidirectional=BIDIRECTIONAL,
               n_numerical_col=n_numerical_col)

In [140]:
for cat, num, y in valid_loader:
    output=model((cat, num))

In [141]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-1.7380e+00,  1.1586e+00, -2.4196e-01, -1.0174e+00, -7.0563e-01],
        [-2.1830e-02, -3.6100e-01,  3.7388e-01,  1.3601e+00, -2.3962e-01],
        [ 2.5806e-01, -6.6231e-01, -2.6955e+00,  1.2209e+00, -4.9280e-01],
        [-2.3041e+00,  4.5592e-01, -5.1058e-02, -1.4496e-01,  1.1416e-01],
        [-2.2915e-01,  1.8016e+00, -1.3804e+00, -1.5945e+00,  6.4478e-01],
        [ 1.0316e+00,  1.3452e+00, -8.7904e-01,  5.3327e-01,  1.1660e+00],
        [-6.6622e-01, -1.8980e-02,  1.3732e+00,  3.6136e-01, -1.2391e+00],
        [ 1.2305e-01,  5.1021e-01, -2.1508e+00,  9.3051e-01,  3.4882e-01],
        [ 5.2211e-01,  1.9039e-01, -2.8668e-01,  3.6039e-01, -1.3207e+00],
        [ 9.6940e-01, -6.5954e-01, -1.2805e-01, -2.2280e+00, -9.8705e-01],
        [ 7.7842e-01, -2.3574e-01,  8.4581e-01,  1.5345e+00,  5.8064e-01],
        [-1.9255e+00,  1.3701e-02,  1.8040e+00, -1.9138e-01, -9.7363e-01],
        [-6.4060e-01,  1.2618e+00, -8.9480e-01,  1.0585e+00, -1.5477e+00],
   